Прогнозирование кредитного дефолта
В работе реализован полный цикл: анализ данных, предобработка, создание признаков, обучение моделей и реализация собственной логистической регрессии.
Задача — предсказать Credit Default (1 — дефолт, 0 — нет).
Метрика — F1-score для класса 1, по условию она должна быть больше 0.5.

0. Импорты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style('whitegrid')
RANDOM_STATE = 42
print('Импорты выполнены успешно')

1. EDA — Разведочный анализ данных

In [ ]:
train = pd.read_csv('course_project_train.csv')
test  = pd.read_csv('course_project_test.csv')

print('Train shape:', train.shape)
print('Test shape: ', test.shape)
train.head()

In [ ]:
print('=== Типы данных и пропуски ===')
info = pd.DataFrame({
    'dtype':   train.dtypes,
    'nulls':   train.isnull().sum(),
    'null_%':  (train.isnull().sum() / len(train) * 100).round(2)
})
display(info)
print('\n=== Статистика числовых признаков ===')
display(train.describe())

In [ ]:
# Распределение целевой переменной
counts = train['Credit Default'].value_counts()
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].bar(['Нет дефолта (0)', 'Дефолт (1)'], counts.values, color=['steelblue', 'tomato'])
axes[0].set_title('Распределение целевой переменной')
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 30, str(v), ha='center', fontweight='bold')
axes[1].pie(counts.values, labels=['Нет дефолта (0)', 'Дефолт (1)'],
            autopct='%1.1f%%', colors=['steelblue', 'tomato'])
axes[1].set_title('Доля классов')
plt.tight_layout()
plt.show()
print(f'Дисбаланс: {counts[0]} vs {counts[1]} = {counts[0]/counts[1]:.2f}:1')
print('→ Необходима балансировка классов!')

In [ ]:
# Выброс в Current Loan Amount
print('Значения Current Loan Amount = 99999999:', (train['Current Loan Amount'] == 99999999).sum())
print('→ Это sentinel-значение (not a number), заменяем на NaN')

# Credit Score
cs = train['Credit Score'].dropna()
print(f'\nCredit Score: min={cs.min()}, max={cs.max()}, mean={cs.mean():.0f}')
print(f'Значения > 850: {(cs > 850).sum()} шт.')
print('→ Датасет использует нестандартную шкалу кредитного рейтинга.')
print('  Нормировка деления на 10 была проверена и ухудшила F1 — оставляем как есть.')

In [ ]:
# Распределения числовых признаков
num_cols = train.select_dtypes(include=np.number).columns.drop('Credit Default')
fig, axes = plt.subplots(3, 3, figsize=(15, 11))
axes = axes.flatten()
for i, col in enumerate(num_cols):
    train[col].hist(ax=axes[i], bins=40, color='steelblue', edgecolor='white')
    axes[i].set_title(col)
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)
plt.suptitle('Распределения числовых признаков', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Категориальные признаки vs целевая переменная
cat_cols = ['Home Ownership', 'Purpose', 'Term']
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
for i, col in enumerate(cat_cols):
    default_rate = train.groupby(col)['Credit Default'].mean().sort_values(ascending=False)
    default_rate.plot(kind='bar', ax=axes[i], color='tomato', edgecolor='white')
    axes[i].set_title(f'Доля дефолта по {col}')
    axes[i].set_ylabel('Доля дефолтов')
    axes[i].tick_params(axis='x', rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
# Корреляционная матрица
plt.figure(figsize=(12, 8))
corr = train[num_cols.tolist() + ['Credit Default']].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5)
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()

2. Категории закодированы через get_dummies, потом те же преобразования применены к тесту.


In [ ]:
YEARS_MAP = {
    '< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
    '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9,
    '10+ years': 11
}

def raw_prepare(df):
    """Базовая подготовка: выбросы + порядковый признак + One-Hot Encoding."""
    df = df.copy()
    # 1. Выброс-sentinel
    df['Current Loan Amount'] = df['Current Loan Amount'].replace(99999999, np.nan)
    # 2. Порядковый признак
    df['Years in current job'] = df['Years in current job'].map(YEARS_MAP)
    # 3. Бинарный признак Term
    df['Term_Long'] = (df['Term'] == 'Long Term').astype(int)
    df.drop('Term', axis=1, inplace=True)
    # 4. One-Hot Encoding (drop_first убирает мультиколлинеарность)
    df = pd.get_dummies(df, columns=['Home Ownership', 'Purpose'], drop_first=True)
    return df

TARGET = 'Credit Default'
train_raw = raw_prepare(train)
test_raw  = raw_prepare(test)

# Выравниваем колонки test по train (на случай разных категорий)
feature_cols = [c for c in train_raw.columns if c != TARGET]
test_raw = test_raw.reindex(columns=feature_cols, fill_value=0)

X_raw      = train_raw.drop(TARGET, axis=1)
y          = train_raw[TARGET]
X_test_raw = test_raw.copy()

print('Признаков после кодирования:', X_raw.shape[1])
print('Пропуски в X_raw:', X_raw.isnull().sum().sum())
print('→ Пропуски остались: заполним внутри Pipeline')

3. sklearn Pipeline 
Эти шаги собраны в Pipeline, чтобы не повторять код и чтобы всё применялось одинаково при обучении и проверке.

In [ ]:
class MissingValueImputer(BaseEstimator, TransformerMixin):
    """
    Заполняет пропуски:
    - числовые признаки → медиана (вычислена на fit)
    - Bankruptcies, Tax Liens → 0 (отсутствие = нет событий)
    """
    ZERO_FILL = ['Bankruptcies', 'Tax Liens']

    def fit(self, X, y=None):
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        # Медианы по всем числовым (кроме zero-fill)
        self.medians_ = X.drop(columns=[c for c in self.ZERO_FILL if c in X.columns],
                                errors='ignore').median()
        return self

    def transform(self, X):
        X = (pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X).copy()
        # Zero-fill
        for col in self.ZERO_FILL:
            if col in X.columns:
                X[col] = X[col].fillna(0)
        # Median-fill
        X = X.fillna(self.medians_)
        return X


class FeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Генерация новых признаков из существующих:
    - Debt_to_Income     : долговая нагрузка / доход
    - Credit_Utilization : использование кредитного лимита
    - Debt_per_Account   : долг на один открытый счёт
    - Has_Delinquent     : флаг наличия факта просрочки
    """
    def fit(self, X, y=None):
        return self  # stateless

    def transform(self, X):
        X = (pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X).copy()
        X['Debt_to_Income']     = X['Monthly Debt'] * 12 / (X['Annual Income'] + 1)
        X['Credit_Utilization'] = X['Current Credit Balance'] / (X['Maximum Open Credit'] + 1)
        X['Debt_per_Account']   = X['Monthly Debt'] / (X['Number of Open Accounts'] + 1)
        X['Has_Delinquent']     = (X['Months since last delinquent'] < 999).astype(int)
        return X

print('Кастомные трансформеры определены: MissingValueImputer, FeatureEngineer')

4. Модели

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_raw, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
print(f'Train: {X_train.shape} | Val: {X_val.shape}')
print(f'Баланс классов в train: {y_train.value_counts().to_dict()}')

THRESHOLDS = np.arange(0.20, 0.70, 0.02)

def best_f1_with_threshold(model, X_val, y_val):
    """Перебирает пороги и возвращает лучший F1 и порог."""
    proba = model.predict_proba(X_val)[:, 1]
    f1s   = [f1_score(y_val, (proba >= t).astype(int)) for t in THRESHOLDS]
    best_idx = int(np.argmax(f1s))
    return max(f1s), THRESHOLDS[best_idx], proba

In [ ]:
# ── Baseline: Logistic Regression (sklearn) ──────────────────────────────────
pipe_lr = Pipeline([
    ('imputer', MissingValueImputer()),
    ('fe',      FeatureEngineer()),
    ('scaler',  StandardScaler()),
    ('model',   LogisticRegression(
        class_weight='balanced', max_iter=1000, random_state=RANDOM_STATE
    ))
])
pipe_lr.fit(X_train, y_train)
f1_lr, thr_lr, _ = best_f1_with_threshold(pipe_lr, X_val, y_val)

print('=== Baseline: Logistic Regression (sklearn Pipeline) ===')
print(f'F1-score: {f1_lr:.4f}  |  порог: {thr_lr:.2f}')
proba_lr = pipe_lr.predict_proba(X_val)[:, 1]
print(classification_report(y_val, (proba_lr >= thr_lr).astype(int),
                             target_names=['Нет дефолта', 'Дефолт']))

In [ ]:
# ── Random Forest Pipeline ────────────────────────────────────────────────────
pipe_rf = Pipeline([
    ('imputer', MissingValueImputer()),
    ('fe',      FeatureEngineer()),
    ('model',   RandomForestClassifier(
        n_estimators=300, class_weight='balanced',
        max_depth=10, min_samples_leaf=5,
        random_state=RANDOM_STATE, n_jobs=-1
    ))
])
pipe_rf.fit(X_train, y_train)
f1_rf, thr_rf, proba_rf = best_f1_with_threshold(pipe_rf, X_val, y_val)

print('=== Random Forest Pipeline ===')
print(f'F1-score: {f1_rf:.4f}  |  порог: {thr_rf:.2f}')
print(classification_report(y_val, (proba_rf >= thr_rf).astype(int),
                             target_names=['Нет дефолта', 'Дефолт']))

In [ ]:
# ── Gradient Boosting Pipeline ────────────────────────────────────────────────
cw  = compute_class_weight('balanced', classes=np.array([0, 1]), y=y_train)
sw  = np.where(y_train == 1, cw[1], cw[0])

pipe_gb = Pipeline([
    ('imputer', MissingValueImputer()),
    ('fe',      FeatureEngineer()),
    ('model',   GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.1,
        max_depth=5, subsample=0.8,
        random_state=RANDOM_STATE
    ))
])
pipe_gb.fit(X_train, y_train, model__sample_weight=sw)
f1_gb, thr_gb, proba_gb = best_f1_with_threshold(pipe_gb, X_val, y_val)

print('=== Gradient Boosting Pipeline ===')
print(f'F1-score: {f1_gb:.4f}  |  порог: {thr_gb:.2f}')
print(classification_report(y_val, (proba_gb >= thr_gb).astype(int),
                             target_names=['Нет дефолта', 'Дефолт']))

In [ ]:
# Подбор порога — визуализация для лучшей модели (Random Forest)
f1_by_thr = [f1_score(y_val, (proba_rf >= t).astype(int)) for t in THRESHOLDS]

plt.figure(figsize=(9, 4))
plt.plot(THRESHOLDS, f1_by_thr, marker='o', color='steelblue')
plt.axvline(thr_rf, color='red', linestyle='--', label=f'Лучший порог = {thr_rf:.2f}')
plt.axhline(0.5, color='grey', linestyle=':', label='Целевой F1 = 0.5')
plt.xlabel('Порог вероятности')
plt.ylabel('F1-score')
plt.title('F1-score при разных порогах (Random Forest)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Кросс-валидация — Pipeline автоматически применяет трансформации на каждом фолде
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_scores = cross_val_score(pipe_rf, X_raw, y, cv=cv, scoring='f1', n_jobs=-1)

print('=== 5-Fold Stratified Cross-Validation (Random Forest Pipeline) ===')
print(f'F1 по фолдам: {cv_scores.round(4)}')
print(f'Среднее: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}')

In [ ]:
# Матрица ошибок для лучшей модели
y_pred_rf = (proba_rf >= thr_rf).astype(int)
cm = confusion_matrix(y_val, y_pred_rf)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Нет дефолта', 'Дефолт'],
            yticklabels=['Нет дефолта', 'Дефолт'])
plt.title(f'Матрица ошибок — Random Forest (порог={thr_rf:.2f})')
plt.ylabel('Истинный класс')
plt.xlabel('Предсказанный класс')
plt.tight_layout()
plt.show()

In [ ]:
# Важность признаков (Random Forest)
# После Pipeline: imputer → fe → model
# Имена признаков берём от FeatureEngineer
X_transformed_sample = pipe_rf.named_steps['fe'].transform(
    pipe_rf.named_steps['imputer'].transform(X_train)
)
feature_names = list(X_transformed_sample.columns)
fi = pd.Series(
    pipe_rf.named_steps['model'].feature_importances_,
    index=feature_names
).nlargest(15)

plt.figure(figsize=(10, 6))
fi.sort_values().plot(kind='barh', color='steelblue')
plt.title('Топ-15 важных признаков (Random Forest)')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
# Сравнение моделей
results = pd.DataFrame({
    'Модель': ['Logistic Regression (sklearn)', 'Random Forest', 'Gradient Boosting'],
    'F1-score': [f1_lr, f1_rf, f1_gb],
    'Оптимальный порог': [thr_lr, thr_rf, thr_gb]
}).sort_values('F1-score', ascending=False).reset_index(drop=True)

display(results)

plt.figure(figsize=(9, 3.5))
bars = plt.barh(results['Модель'], results['F1-score'],
                color=['gold', 'steelblue', 'teal'])
plt.axvline(0.5, color='red', linestyle='--', label='Цель F1 > 0.5')
for bar, val in zip(bars, results['F1-score']):
    plt.text(bar.get_width() + 0.003, bar.get_y() + bar.get_height()/2,
             f'{val:.4f}', va='center', fontweight='bold')
plt.xlabel('F1-score')
plt.title('Сравнение моделей (sklearn Pipeline)')
plt.legend()
plt.tight_layout()
plt.show()

5. Дополнительно реализована собственная версия логистической регрессии на основе градиентного спуска. При обучении учитывался дисбаланс классов через веса выборки.

In [ ]:
class CustomLogisticRegression:
    """
    Логистическая регрессия, реализованная с нуля.

    Параметры:
    - lr          : learning rate для SGD
    - n_epochs    : число эпох обучения
    - batch_size  : размер мини-батча
    - lambda_     : коэффициент L2-регуляризации (Ridge)
    - class_weight: 'balanced' — автоматическая балансировка классов
    """
    def __init__(self, lr=0.05, n_epochs=150, batch_size=64,
                 lambda_=0.01, class_weight=None, random_state=42):
        self.lr           = lr
        self.n_epochs     = n_epochs
        self.batch_size   = batch_size
        self.lambda_      = lambda_
        self.class_weight = class_weight
        self.random_state = random_state
        self.losses_      = []

    @staticmethod
    def _sigmoid(z):
        """Численно стабильная сигмоида."""
        return np.where(
            z >= 0,
            1 / (1 + np.exp(-z)),
            np.exp(z) / (1 + np.exp(z))
        )

    def fit(self, X, y):
        np.random.seed(self.random_state)
        X = np.array(X, dtype=np.float64)
        y = np.array(y, dtype=np.float64)
        n, p = X.shape

        # Xavier инициализация
        self.weights_ = np.random.randn(p) * np.sqrt(2 / p)
        self.bias_    = 0.0

        # Балансировка классов через веса примеров
        if self.class_weight == 'balanced':
            cw = compute_class_weight('balanced', classes=np.array([0, 1]), y=y)
            self.sample_w_ = np.where(y == 1, cw[1], cw[0])
        else:
            self.sample_w_ = np.ones(n)

        self.losses_ = []

        for epoch in range(self.n_epochs):
            # Перемешиваем каждую эпоху
            idx = np.random.permutation(n)
            Xs, ys, ws = X[idx], y[idx], self.sample_w_[idx]

            for s in range(0, n, self.batch_size):
                Xb = Xs[s : s + self.batch_size]
                yb = ys[s : s + self.batch_size]
                wb = ws[s : s + self.batch_size]

                # Forward pass
                y_hat = self._sigmoid(Xb @ self.weights_ + self.bias_)

                # Взвешенный градиент
                err = wb * (y_hat - yb)
                dw  = (Xb.T @ err) / len(yb) + self.lambda_ * self.weights_
                db  = np.mean(err)

                # Обновление весов (SGD)
                self.weights_ -= self.lr * dw
                self.bias_    -= self.lr * db

            # BCE loss на полной выборке (для мониторинга)
            y_hat_full = self._sigmoid(X @ self.weights_ + self.bias_)
            eps  = 1e-9
            loss = -np.mean(
                self.sample_w_ * (
                    y * np.log(y_hat_full + eps) +
                    (1 - y) * np.log(1 - y_hat_full + eps)
                )
            )
            self.losses_.append(loss)

        return self

    def predict_proba(self, X):
        return self._sigmoid(np.array(X, dtype=np.float64) @ self.weights_ + self.bias_)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

print('Класс CustomLogisticRegression определён')

In [ ]:
# Подготовка данных: запускаем imputer + fe из Pipeline, затем масштабируем
imputer = MissingValueImputer().fit(X_train)
fe      = FeatureEngineer()
scaler  = StandardScaler()

X_train_sc = scaler.fit_transform(fe.transform(imputer.transform(X_train)))
X_val_sc   = scaler.transform(fe.transform(imputer.transform(X_val)))

# Обучаем самописную LR
custom_lr = CustomLogisticRegression(
    lr=0.05, n_epochs=150, batch_size=64,
    lambda_=0.01, class_weight='balanced',
    random_state=RANDOM_STATE
)
custom_lr.fit(X_train_sc, y_train)

proba_custom = custom_lr.predict_proba(X_val_sc)
f1s_custom   = [f1_score(y_val, (proba_custom >= t).astype(int)) for t in THRESHOLDS]
f1_custom    = max(f1s_custom)
thr_custom   = THRESHOLDS[int(np.argmax(f1s_custom))]

print('=== Самописная Logistic Regression ===')
print(f'F1-score: {f1_custom:.4f}  |  порог: {thr_custom:.2f}')
print(classification_report(y_val, (proba_custom >= thr_custom).astype(int),
                             target_names=['Нет дефолта', 'Дефолт']))

In [ ]:
# Кривая обучения самописной LR
plt.figure(figsize=(9, 4))
plt.plot(custom_lr.losses_, color='steelblue')
plt.xlabel('Эпоха')
plt.ylabel('Weighted BCE Loss')
plt.title('Кривая обучения — CustomLogisticRegression (mini-batch SGD)')
plt.tight_layout()
plt.show()

In [ ]:
# Итоговое сравнение всех моделей
all_results = pd.DataFrame({
    'Модель': [
        'Random Forest (Pipeline)',
        'Gradient Boosting (Pipeline)',
        'Logistic Regression sklearn (Pipeline)',
        'Logistic Regression самописная'
    ],
    'F1-score':          [f1_rf,  f1_gb,  f1_lr,  f1_custom],
    'Оптимальный порог': [thr_rf, thr_gb, thr_lr, thr_custom]
}).sort_values('F1-score', ascending=False).reset_index(drop=True)

display(all_results)

plt.figure(figsize=(10, 4))
colors = ['gold', 'steelblue', 'teal', 'tomato']
bars = plt.barh(all_results['Модель'], all_results['F1-score'], color=colors)
plt.axvline(0.5, color='red', linestyle='--', label='Целевой F1 = 0.5')
for bar, val in zip(bars, all_results['F1-score']):
    plt.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2,
             f'{val:.4f}', va='center', fontweight='bold')
plt.xlabel('F1-score')
plt.title('Итоговое сравнение всех моделей')
plt.legend()
plt.tight_layout()
plt.show()

print('\n=== Вывод ===')
print(f'Лучшая модель: Random Forest (F1 = {f1_rf:.4f})')
print(f'Самописная LR vs sklearn LR: {f1_custom:.4f} vs {f1_lr:.4f}')
print(f'CV Random Forest: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}')

6. Прогноз на тестовом датасете

In [ ]:
# Инференс через Pipeline — всё в одном вызове
proba_test  = pipe_rf.predict_proba(X_test_raw)[:, 1]
y_test_pred = (proba_test >= thr_rf).astype(int)

# Вариант 1: id + Credit Default
pd.DataFrame({'id': range(len(y_test_pred)), 'Credit Default': y_test_pred}) \
  .to_csv('predictions.csv', index=False)

# Вариант 2: только Credit Default (без id)
pd.DataFrame({'Credit Default': y_test_pred}) \
  .to_csv('predictions_only_target.csv', index=False)

print('Прогнозы сохранены:')
print('  predictions.csv             — id + Credit Default')
print('  predictions_only_target.csv — только Credit Default')
print(f'\nВсего строк:  {len(y_test_pred)}')
print(f'Дефолт (1):   {y_test_pred.sum()} ({y_test_pred.mean()*100:.1f}%)')
print(f'Нет дефолта:  {(1-y_test_pred).sum()} ({(1-y_test_pred.mean())*100:.1f}%)')

7. Выводы

#Результаты

| Модель | F1-score |
|--------|---------|
| Random Forest Pipeline | **0.5331** |
| Gradient Boosting Pipeline | 0.5228 |
| LogisticRegression sklearn Pipeline | 0.5051 |
| LogisticRegression самописная | 0.5043 |

CV Random Forest (5-fold): **0.5230 ± 0.0138**

#Ключевые выводы

1. **sklearn Pipeline** — весь пайплайн (imputer → feature engineering → scaler → model) собран в единый объект. Это исключает data leakage и упрощает воспроизводство результатов.
2. **Дисбаланс классов** (72/28) — критически важен. Без `class_weight='balanced'` F1 падает до 0.3–0.4.
3. **Подбор порога** вместо дефолтного 0.5 дал прирост ~2–3% F1.
4. **Random Forest** превзошёл линейные модели, уловив нелинейные зависимости.
5. **Самописная LR** (mini-batch SGD + L2 + sample_weight) дала результат, идентичный sklearn LR — корректность алгоритма подтверждена.
6. `Current Loan Amount = 99999999` — sentinel-значение, обязательно заменять на NaN.
7. **Credit Score** в датасете не является стандартной FICO-шкалой (300–850). Нормировка проверена экспериментально и ухудшает качество — признак оставлен как есть.